# MVP – Pets na Cesta de Consumo 
## Pipeline completo (Bronze → Silver → Gold → Análise)

Este notebook reúne **todas as etapas do MVP** em um único fluxo, por limitação de criação de múltiplos notebooks na Databricks Community Edition.  

Aqui estão consolidadas as três partes originalmente planejadas:

1. **Busca e Coleta de Dados (Bronze / Silver)**  
   - Leitura do workbook do ONS (Family spending – FYE 2024, tabela A6);  
   - Criação da tabela bruta `family_spending_a6`;  
   - Limpeza e padronização em `family_spending_a6_clean`;  
   - Transformação do formato *wide* (decis em colunas) para *long* (decis em linhas) na view `vw_despesa_long`;  
   - Cálculo do gasto total por decil e da participação da categoria no orçamento em `vw_despesa_com_total`.

2. **Modelagem e Carga da Camada Gold (Esquema Estrela)**  
   - Criação das dimensões: `dim_renda`, `dim_categoria_consumo`, `dim_tempo`, `dim_geografia`;  
   - Criação da tabela fato `fato_despesa_familiar`, com gasto médio semanal, gasto total do decil e participação da categoria no orçamento.

3. **Análise de Teoria do Consumidor (Gastos com Pets)**  
   - Análise de qualidade de dados (valores mínimos, máximos e nulos);  
   - Cálculo da participação dos gastos com **pets** no orçamento por decil de renda;  
   - Comparação entre **Pets** e **Alimentação** (Lei de Engel);  
   - Geração de tabelas e gráficos usados na documentação do MVP.

> **Observação:** embora o plano original previsse três notebooks (`01_busca_coleta`, `02_modelagem_carga` e `03_analise_consumidor_pets`), todas as etapas foram integradas neste único notebook devido à limitação da conta gratuita do Databricks. A organização em seções internas mantém a separação lógica entre as fases do trabalho.

---



## 1. Configuração inicial (catálogo / schema)

Nesta seção configuramos o catálogo e o schema padrão onde as tabelas e views do MVP serão criadas.


In [0]:
USE CATALOG workspace;
USE SCHEMA default;

## 2. Camada Bronze – Tabela original `family_spending_a6`

A tabela `family_spending_a6` foi criada a partir do upload do arquivo do ONS (tabela A6 do workbook 1 – FYE 2024).  
Aqui apenas inspecionamos alguns registros para confirmar a estrutura.


In [0]:
SELECT *
FROM family_spending_a6
LIMIT 10;

code,description,d1_lowest,d2_second,d3_third,d4_fourth,d5_fifth,d6_sixth,d7_seventh,d8_eighth,d9_ninth,d10_highest,all_households
1,Food & non-alcoholic drinks,38.1,48.3,57.0,62.5,68.6,71.3,76.7,88.7,93.2,100.9,70.5
1.1,Food,35.2,44.2,52.4,56.9,62.4,65.2,69.9,80.6,84.3,91.9,64.3
1.1.1,"Bread, rice and cereals",3.3,4.0,5.2,5.2,5.9,6.6,6.6,8.2,8.2,8.6,6.2
1.1.2,Pasta products,0.3,0.3,0.5,0.4,0.6,0.6,0.7,0.7,0.7,0.8,0.6
1.1.3,"Buns, cakes, biscuits etc.",2.6,3.6,4.1,4.2,4.8,4.7,5.4,6.3,6.4,6.4,4.9
1.1.4,Pastry (savoury),0.6,0.5,0.7,1.0,1.1,1.2,1.2,1.5,2.0,1.3,1.1
1.1.5,"Beef (fresh, chilled or frozen)",0.8,1.2,1.7,2.0,2.1,2.0,2.4,2.2,2.1,3.5,2.0
1.1.6,"Pork (fresh, chilled or frozen)",0.3,0.4,0.6,0.5,0.6,0.6,0.6,0.7,0.8,0.7,0.6
1.1.7,"Lamb (fresh, chilled or frozen)",0.3,0.4,0.6,0.7,0.4,0.6,0.6,0.6,0.4,0.9,0.5
1.1.8,"Poultry (fresh, chilled or frozen)",1.0,1.2,2.0,1.8,2.7,2.6,2.7,3.4,3.6,3.7,2.5


## 3. Camada Silver – Limpeza e seleção de colunas (`family_spending_a6_clean`)

Nesta etapa criamos a tabela `family_spending_a6_clean`, que:
- seleciona apenas as colunas relevantes da tabela A6;
- converte os valores de despesa por decil e o total para tipo numérico (`DOUBLE`).


In [0]:
CREATE OR REPLACE TABLE family_spending_a6_clean AS
SELECT
  code,
  description,
  `d1_lowest`      AS d1_lowest,
  `d2_second`      AS d2_second,
  `d3_third`       AS d3_third,
  `d4_fourth`      AS d4_fourth,
  `d5_fifth`       AS d5_fifth,
  `d6_sixth`       AS d6_sixth,
  `d7_seventh`     AS d7_seventh,
  `d8_eighth`      AS d8_eighth,
  `d9_ninth`       AS d9_ninth,
  `d10_highest`    AS d10_highest,
  `all_households` AS all_households
FROM family_spending_a6;


num_affected_rows,num_inserted_rows


In [0]:
SELECT *
FROM family_spending_a6_clean
LIMIT 10;

code,description,d1_lowest,d2_second,d3_third,d4_fourth,d5_fifth,d6_sixth,d7_seventh,d8_eighth,d9_ninth,d10_highest,all_households
1,Food & non-alcoholic drinks,38.1,48.3,57.0,62.5,68.6,71.3,76.7,88.7,93.2,100.9,70.5
1.1,Food,35.2,44.2,52.4,56.9,62.4,65.2,69.9,80.6,84.3,91.9,64.3
1.1.1,"Bread, rice and cereals",3.3,4.0,5.2,5.2,5.9,6.6,6.6,8.2,8.2,8.6,6.2
1.1.2,Pasta products,0.3,0.3,0.5,0.4,0.6,0.6,0.7,0.7,0.7,0.8,0.6
1.1.3,"Buns, cakes, biscuits etc.",2.6,3.6,4.1,4.2,4.8,4.7,5.4,6.3,6.4,6.4,4.9
1.1.4,Pastry (savoury),0.6,0.5,0.7,1.0,1.1,1.2,1.2,1.5,2.0,1.3,1.1
1.1.5,"Beef (fresh, chilled or frozen)",0.8,1.2,1.7,2.0,2.1,2.0,2.4,2.2,2.1,3.5,2.0
1.1.6,"Pork (fresh, chilled or frozen)",0.3,0.4,0.6,0.5,0.6,0.6,0.6,0.7,0.8,0.7,0.6
1.1.7,"Lamb (fresh, chilled or frozen)",0.3,0.4,0.6,0.7,0.4,0.6,0.6,0.6,0.4,0.9,0.5
1.1.8,"Poultry (fresh, chilled or frozen)",1.0,1.2,2.0,1.8,2.7,2.6,2.7,3.4,3.6,3.7,2.5


## 4. Camada Silver – Transformação wide → long (`vw_despesa_long`)

A tabela A6 original apresenta os decis de renda em colunas (`d1_lowest` … `d10_highest`).  
Aqui criamos a view `vw_despesa_long`, que coloca os dados em formato analítico *long*:

- uma linha por combinação **[categoria × decil de renda]**;
- coluna `decil_renda` (1 a 10);
- coluna `gasto_medio_semanal` com o valor correspondente ao decil.


In [0]:
CREATE OR REPLACE VIEW vw_despesa_long AS
SELECT
  2024 AS ano,              -- tudo é FYE 2024
  code,
  description,
  decil_renda,
  gasto_medio_semanal
FROM family_spending_a6_clean
LATERAL VIEW STACK (
  10,
  1, d1_lowest,
  2, d2_second,
  3, d3_third,
  4, d4_fourth,
  5, d5_fifth,
  6, d6_sixth,
  7, d7_seventh,
  8, d8_eighth,
  9, d9_ninth,
  10, d10_highest
) AS decil_renda, gasto_medio_semanal;


In [0]:
SELECT *
FROM vw_despesa_long
LIMIT 10;


ano,code,description,decil_renda,gasto_medio_semanal
2024,1,Food & non-alcoholic drinks,1,38.1
2024,1.1,Food,1,35.2
2024,1.1.1,"Bread, rice and cereals",1,3.3
2024,1.1.2,Pasta products,1,0.3
2024,1.1.3,"Buns, cakes, biscuits etc.",1,2.6
2024,1.1.4,Pastry (savoury),1,0.6
2024,1.1.5,"Beef (fresh, chilled or frozen)",1,0.8
2024,1.1.6,"Pork (fresh, chilled or frozen)",1,0.3
2024,1.1.7,"Lamb (fresh, chilled or frozen)",1,0.3
2024,1.1.8,"Poultry (fresh, chilled or frozen)",1,1.0


## 5. Camada Silver – Cálculo do gasto total e participação no orçamento (`vw_despesa_com_total`)

A partir de `vw_despesa_long`, criamos a view `vw_despesa_com_total`, que adiciona:

- `gasto_medio_total_semanal` (coluna `all_households`);  
- `participacao_orcamento` = gasto da categoria / gasto total do decil.


In [0]:
CREATE OR REPLACE VIEW vw_despesa_com_total AS
WITH base AS (
  SELECT *
  FROM vw_despesa_long
),
totais AS (
  SELECT
    ano,
    decil_renda,
    SUM(gasto_medio_semanal) AS gasto_medio_total_semanal
  FROM base
  GROUP BY ano, decil_renda
)
SELECT
  b.ano,
  b.code,
  b.description,
  b.decil_renda,
  b.gasto_medio_semanal,
  t.gasto_medio_total_semanal,
  b.gasto_medio_semanal / t.gasto_medio_total_semanal AS participacao_orcamento
FROM base b
JOIN totais t
  ON b.ano = t.ano
 AND b.decil_renda = t.decil_renda;


In [0]:
SELECT *
FROM vw_despesa_com_total
LIMIT 10;

ano,code,description,decil_renda,gasto_medio_semanal,gasto_medio_total_semanal,participacao_orcamento
2024,1,Food & non-alcoholic drinks,1,38.1,1487.0000000000002,0.02562205783456624
2024,1.1,Food,1,35.2,1487.0000000000002,0.023671822461331538
2024,1.1.1,"Bread, rice and cereals",1,3.3,1487.0000000000002,0.002219233355749831
2024,1.1.2,Pasta products,1,0.3,1487.0000000000002,2.0174848688634832E-4
2024,1.1.3,"Buns, cakes, biscuits etc.",1,2.6,1487.0000000000002,0.0017484868863483522
2024,1.1.4,Pastry (savoury),1,0.6,1487.0000000000002,4.0349697377269663E-4
2024,1.1.5,"Beef (fresh, chilled or frozen)",1,0.8,1487.0000000000002,5.379959650302622E-4
2024,1.1.6,"Pork (fresh, chilled or frozen)",1,0.3,1487.0000000000002,2.0174848688634832E-4
2024,1.1.7,"Lamb (fresh, chilled or frozen)",1,0.3,1487.0000000000002,2.0174848688634832E-4
2024,1.1.8,"Poultry (fresh, chilled or frozen)",1,1.0,1487.0000000000002,6.724949562878277E-4


## 6. Camada Gold – Esquema estrela (dimensões e tabela fato)

Nesta seção criamos as tabelas da camada Gold, seguindo o modelo em esquema estrela:

- Dimensões: `dim_renda`, `dim_categoria_consumo`, `dim_tempo`, `dim_geografia`;  
- Tabela fato: `fato_despesa_familiar`, que consolida gasto médio semanal, gasto total e participação no orçamento.


In [0]:
CREATE OR REPLACE TABLE dim_renda AS
SELECT
  ROW_NUMBER() OVER (ORDER BY decil_renda) AS id_renda,
  decil_renda,
  CONCAT(
    'Decil ',
    decil_renda,
    ' - ',
    CASE 
      WHEN decil_renda = 1  THEN '10% mais pobres'
      WHEN decil_renda = 10 THEN '10% mais ricos'
      ELSE 'faixa intermediária'
    END
  ) AS descricao_decil,
  'renda bruta por decil' AS tipo_renda
FROM (
  SELECT DISTINCT decil_renda
  FROM vw_despesa_long
);


num_affected_rows,num_inserted_rows


In [0]:
SELECT * FROM dim_renda ORDER BY decil_renda;


id_renda,decil_renda,descricao_decil,tipo_renda
1,1,Decil 1 - 10% mais pobres,renda bruta por decil
2,2,Decil 2 - faixa intermediária,renda bruta por decil
3,3,Decil 3 - faixa intermediária,renda bruta por decil
4,4,Decil 4 - faixa intermediária,renda bruta por decil
5,5,Decil 5 - faixa intermediária,renda bruta por decil
6,6,Decil 6 - faixa intermediária,renda bruta por decil
7,7,Decil 7 - faixa intermediária,renda bruta por decil
8,8,Decil 8 - faixa intermediária,renda bruta por decil
9,9,Decil 9 - faixa intermediária,renda bruta por decil
10,10,Decil 10 - 10% mais ricos,renda bruta por decil


In [0]:
CREATE OR REPLACE TABLE dim_categoria_consumo AS
SELECT
  ROW_NUMBER() OVER (ORDER BY code, description) AS id_categoria,
  code        AS codigo_categoria,
  description AS descricao_categoria,
  CASE 
    WHEN LOWER(description) LIKE '%pet%' THEN 1
    ELSE 0
  END AS flag_pet
FROM (
  SELECT DISTINCT code, description
  FROM vw_despesa_long
);


num_affected_rows,num_inserted_rows


In [0]:
SELECT *
FROM dim_categoria_consumo
ORDER BY flag_pet DESC, descricao_categoria
LIMIT 20;


id_categoria,codigo_categoria,descricao_categoria,flag_pet
148,7.2.2,"Petrol, diesel and other motor oils",1
171,9.3.5,Pets and pet food,1
187,gardens and pets,gardens and pets,1
184,Total expenditure,null,0
115,3.1.9,Accessories,0
59,11.2,Accommodation services,0
118,4.1,Actual rentals for housing,0
94,2,"Alcoholic drink, tobacco & narcotics",0
95,2.1,Alcoholic drinks,0
55,11.1.2,Alcoholic drinks (away from home),0


In [0]:
CREATE OR REPLACE TABLE dim_tempo AS
SELECT
  1        AS id_tempo,
  2024     AS ano,
  'FYE 2024' AS periodo_label,
  DATE('2023-04-01') AS ano_inicio,
  DATE('2024-03-31') AS ano_fim;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE dim_geografia AS
SELECT
  1               AS id_geografia,
  'UK total'      AS regiao,
  'Agregado Reino Unido' AS descricao_geografia;


num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE fato_despesa_familiar AS
SELECT
  ROW_NUMBER() OVER (ORDER BY d.ano, d.decil_renda, c.id_categoria) AS id_fato,
  r.id_renda,
  c.id_categoria,
  t.id_tempo,
  g.id_geografia,
  d.gasto_medio_semanal,
  d.gasto_medio_total_semanal,
  d.participacao_orcamento
FROM vw_despesa_com_total d
JOIN dim_renda r
  ON d.decil_renda = r.decil_renda
JOIN dim_categoria_consumo c
  ON d.code = c.codigo_categoria
 AND d.description = c.descricao_categoria
JOIN dim_tempo t
  ON d.ano = t.ano
JOIN dim_geografia g
  ON g.id_geografia = 1;


num_affected_rows,num_inserted_rows


In [0]:
SELECT *
FROM fato_despesa_familiar
LIMIT 10;

id_fato,id_renda,id_categoria,id_tempo,id_geografia,gasto_medio_semanal,gasto_medio_total_semanal,participacao_orcamento
1,1,7,1,1,38.1,1487.0000000000002,0.02562205783456624
2,1,8,1,1,255.1,1487.0000000000002,0.17155346334902485
3,1,9,1,1,35.2,1487.0000000000002,0.023671822461331538
4,1,10,1,1,3.3,1487.0000000000002,0.002219233355749831
5,1,11,1,1,5.1,1487.0000000000002,0.0034297242770679214
6,1,12,1,1,1.7,1487.0000000000002,0.001143241425689307
7,1,13,1,1,1.5,1487.0000000000002,0.0010087424344317416
8,1,14,1,1,1.2,1487.0000000000002,8.069939475453933E-4
9,1,15,1,1,0.7,1487.0000000000002,4.707464694014794E-4
10,1,16,1,1,1.4,1487.0000000000002,9.414929388029588E-4


In [0]:
WITH fato_join AS (
  SELECT
    f.*,
    r.decil_renda,
    c.descricao_categoria,
    c.flag_pet
  FROM fato_despesa_familiar f
  JOIN dim_renda r        ON f.id_renda = r.id_renda
  JOIN dim_categoria_consumo c ON f.id_categoria = c.id_categoria
)
SELECT
  decil_renda,
  SUM(CASE WHEN flag_pet = 1 THEN participacao_orcamento ELSE 0 END) AS share_pets
FROM fato_join
GROUP BY decil_renda
ORDER BY decil_renda;


decil_renda,share_pets
1,0.011230665770006724
2,0.012143253405658635
3,0.012924978926664797
4,0.013587730360847806
5,0.015350120731286651
6,0.013676615283699574
7,0.013131883648686813
8,0.013392519357360841
9,0.01347369320122993
10,0.011559439692574683


## 7. Análise – Qualidade dos dados

Antes da interpretação econômica, avaliamos a qualidade dos dados na tabela fato:
- presença de valores nulos;
- faixas mínimas e máximas de gasto e participação.


In [0]:
SELECT
  SUM(CASE WHEN gasto_medio_semanal      IS NULL THEN 1 ELSE 0 END) AS n_nulos_gasto_medio,
  SUM(CASE WHEN gasto_medio_total_semanal IS NULL THEN 1 ELSE 0 END) AS n_nulos_gasto_total,
  SUM(CASE WHEN participacao_orcamento   IS NULL THEN 1 ELSE 0 END) AS n_nulos_share
FROM fato_despesa_familiar;


n_nulos_gasto_medio,n_nulos_gasto_total,n_nulos_share
197,0,197


In [0]:
USE CATALOG workspace;
USE SCHEMA default;

CREATE OR REPLACE VIEW vw_fato_join AS
SELECT
  f.*,
  r.decil_renda,
  c.descricao_categoria,
  c.flag_pet
FROM fato_despesa_familiar f
JOIN dim_renda r
  ON f.id_renda = r.id_renda
JOIN dim_categoria_consumo c
  ON f.id_categoria = c.id_categoria;


In [0]:
SELECT *
FROM vw_fato_join
LIMIT 10;

id_fato,id_renda,id_categoria,id_tempo,id_geografia,gasto_medio_semanal,gasto_medio_total_semanal,participacao_orcamento,decil_renda,descricao_categoria,flag_pet
1,1,7,1,1,38.1,1487.0000000000002,0.02562205783456624,1,Food & non-alcoholic drinks,0
2,1,8,1,1,255.1,1487.0000000000002,0.17155346334902485,1,All expenditure groups,0
3,1,9,1,1,35.2,1487.0000000000002,0.023671822461331538,1,Food,0
4,1,10,1,1,3.3,1487.0000000000002,0.002219233355749831,1,"Bread, rice and cereals",0
5,1,11,1,1,5.1,1487.0000000000002,0.0034297242770679214,1,Other meat and meat preparations,0
6,1,12,1,1,1.7,1487.0000000000002,0.001143241425689307,1,Fish and fish products,0
7,1,13,1,1,1.5,1487.0000000000002,0.0010087424344317416,1,Milk,0
8,1,14,1,1,1.2,1487.0000000000002,8.069939475453933E-4,1,Cheese and curd,0
9,1,15,1,1,0.7,1487.0000000000002,4.707464694014794E-4,1,Eggs,0
10,1,16,1,1,1.4,1487.0000000000002,9.414929388029588E-4,1,Other milk products,0


In [0]:
SELECT
  decil_renda,
  descricao_categoria,
  gasto_medio_semanal,
  participacao_orcamento
FROM vw_fato_join
WHERE gasto_medio_semanal IS NULL
LIMIT 20;


decil_renda,descricao_categoria,gasto_medio_semanal,participacao_orcamento
3,Dried vegetables,null,null
10,Education,null,null
10,Education fees,null,null
2,Contract catering (food) and canteens,null,null
2,Holiday abroad,null,null
10,Room hire,null,null
6,Social protection,null,null
1,Other services and professional fees,null,null
5,Holiday spending,null,null
10,"Money, cash gifts given to children",null,null


In [0]:
SELECT
  decil_renda,
  SUM(
    CASE 
      WHEN flag_pet = 1 THEN COALESCE(participacao_orcamento, 0)
      ELSE 0
    END
  ) AS share_pets
FROM vw_fato_join
GROUP BY decil_renda
ORDER BY decil_renda;


decil_renda,share_pets
1,0.011230665770006724
2,0.012143253405658635
3,0.012924978926664797
4,0.013587730360847806
5,0.015350120731286651
6,0.013676615283699574
7,0.013131883648686813
8,0.013392519357360841
9,0.01347369320122993
10,0.011559439692574683


In [0]:
CREATE OR REPLACE TABLE analise_share_pets AS
SELECT
  decil_renda,
  SUM(
    CASE 
      WHEN flag_pet = 1 THEN COALESCE(participacao_orcamento, 0)
      ELSE 0
    END
  ) AS share_pets
FROM vw_fato_join
GROUP BY decil_renda
ORDER BY decil_renda;


num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE VIEW vw_fato_grupos AS
SELECT
  decil_renda,
  descricao_categoria,
  flag_pet,
  participacao_orcamento,
  CASE
    WHEN flag_pet = 1 THEN 'Pets'
    WHEN LOWER(descricao_categoria) LIKE '%food%' THEN 'Alimentação'
    ELSE 'Outras'
  END AS grupo_analise
FROM vw_fato_join;


In [0]:
SELECT
  MIN(gasto_medio_semanal)    AS min_gasto,
  MAX(gasto_medio_semanal)    AS max_gasto,
  MIN(participacao_orcamento) AS min_share,
  MAX(participacao_orcamento) AS max_share
FROM fato_despesa_familiar;

min_gasto,max_gasto,min_share,max_share
0.1,961.6,2.1352921079603697E-5,0.17867458606240585


In [0]:
SELECT
  id_renda,
  id_categoria,
  id_tempo,
  id_geografia,
  gasto_medio_semanal,
  gasto_medio_total_semanal,
  participacao_orcamento
FROM fato_despesa_familiar
LIMIT 20;

id_renda,id_categoria,id_tempo,id_geografia,gasto_medio_semanal,gasto_medio_total_semanal,participacao_orcamento
1,7,1,1,38.1,1487.0000000000002,0.02562205783456624
1,8,1,1,255.1,1487.0000000000002,0.17155346334902485
1,9,1,1,35.2,1487.0000000000002,0.023671822461331538
1,10,1,1,3.3,1487.0000000000002,0.002219233355749831
1,11,1,1,5.1,1487.0000000000002,0.0034297242770679214
1,12,1,1,1.7,1487.0000000000002,0.001143241425689307
1,13,1,1,1.5,1487.0000000000002,0.0010087424344317416
1,14,1,1,1.2,1487.0000000000002,8.069939475453933E-4
1,15,1,1,0.7,1487.0000000000002,4.707464694014794E-4
1,16,1,1,1.4,1487.0000000000002,9.414929388029588E-4


In [0]:
SELECT
  decil_renda,
  SUM(
    CASE 
      WHEN flag_pet = 1 THEN COALESCE(participacao_orcamento, 0)
      ELSE 0
    END
  ) AS share_pets
FROM vw_fato_join
GROUP BY decil_renda
ORDER BY decil_renda;


decil_renda,share_pets
1,0.011230665770006724
2,0.012143253405658635
3,0.012924978926664797
4,0.013587730360847806
5,0.015350120731286651
6,0.013676615283699574
7,0.013131883648686813
8,0.013392519357360841
9,0.01347369320122993
10,0.011559439692574683


## 8. Análise – Participação dos gastos com pets por decil de renda

Aqui calculamos a participação total dos gastos com pets no orçamento de cada decil de renda e geramos a curva usada como “curva de Engel aproximada” para pets.


In [0]:
SELECT
  decil_renda,
  100 * share_pets AS share_pets_pct
FROM (
    SELECT
      decil_renda,
      SUM(
        CASE
          WHEN flag_pet = 1 THEN COALESCE(participacao_orcamento, 0)
          ELSE 0
        END
      ) AS share_pets
    FROM vw_fato_join
    GROUP BY decil_renda
) t
ORDER BY decil_renda;


decil_renda,share_pets_pct
1,1.1230665770006725
2,1.2143253405658634
3,1.2924978926664796
4,1.3587730360847805
5,1.5350120731286652
6,1.3676615283699574
7,1.3131883648686813
8,1.3392519357360841
9,1.347369320122993
10,1.1559439692574682


Databricks visualization. Run in Databricks to view.

## 9. Análise – Comparação Pets x Alimentação (Lei de Engel)

Nesta etapa comparamos a participação dos gastos com pets e alimentação no orçamento, por decil de renda, para observar o comportamento esperado pela Lei de Engel.


In [0]:
SELECT
  decil_renda,
  grupo_analise,
  SUM(COALESCE(participacao_orcamento, 0)) AS share
FROM vw_fato_grupos
GROUP BY decil_renda, grupo_analise
ORDER BY decil_renda, grupo_analise;


decil_renda,grupo_analise,share
1,Alimentação,0.05117686617350369
1,Outras,0.7401479488903834
1,Pets,0.011230665770006724
2,Alimentação,0.058867040621340086
2,Outras,0.7261912100104795
2,Pets,0.012143253405658635
3,Alimentação,0.05371358995972653
3,Outras,0.7279198276669477
3,Pets,0.012924978926664797
4,Alimentação,0.05101501159432081
